In [5]:
import numpy as np
import tensorflow as tf

/home/simone/anaconda3/envs/bioch_project/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


What we really want to do is building a *custom estimator* which will simplify our life for results tracking and data visualization. Further improvements might include a distributed Cloud version of it to overcome the important computational cost it requires.

In [6]:
# loading datasets and phreshing folds for evaluations and testing
train_ds, test_ds = np.load("train.npz"), np.load("test.npz")

partition, x_train, y_train = train_ds["partition"], train_ds["X_train"], train_ds["y_train"]
x_test, y_test = test_ds['X_test'], test_ds['y_test']

First attempt, _without_ cross validation

This is the actual training phase:

for cross validation, we train the data on 4 different models by splitting the dataset in 4 subsets.

In [18]:
feature_columns = [tf.feature_column.numeric_column("X", shape=(3569, 1000, 20))]

num_hidden_units = [512, 256, 128]
    
model = tf.estimator.LinearClassifier(feature_columns=feature_columns,
                                       n_classes=10,
                                       model_dir="./checkpoints_tutorial17-1/")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './checkpoints_tutorial17-1/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbfaca40780>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [21]:
# giving an error, but at least we've fixed it
for i in range(1,5):
    x_part, y_part, x_val, y_val = x_train[np.where(partition != i)], y_train[np.where(partition != i)], x_train[np.where(partition == i)], y_train[np.where(partition == i)] 
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(x={'X':x_part},
                                                   y=y_part,
                                                   num_epochs=1,
                                                   shuffle=False)
    
    model.train(input_fn=train_input_fn, steps=10)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.


InvalidArgumentError: Cannot parse tensor from proto: dtype: DT_FLOAT
tensor_shape {
  dim {
    size: 71380000
  }
  dim {
    size: 10
  }
}
float_val: 0.1

	 [[Node: linear/Const = Const[_class=["loc:@linear/linear_model/X/weights/part_0"], dtype=DT_FLOAT, value=<Invalid TensorProto: dtype: DT_FLOAT tensor_shape { dim { size: 71380000 } dim { size: 10 } } float_val: 0.1>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'linear/Const', defined at:
  File "/home/simone/anaconda3/envs/bioch_project/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/simone/anaconda3/envs/bioch_project/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/simone/anaconda3/envs/bioch_project/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/simone/anaconda3/envs/bioch_project/lib/python3.6/site-packages/traitlets/config/application.p

In [8]:
final_test_input_fn = tf.estimator.inputs.numpy_input_fn(x={'X':x_test},
                                                   y=y_test,
                                                   num_epochs=20,
                                                   shuffle=False)

In [ ]:
# the model function
def cnn_clocator(inputs, classes, mode):
    
    conv1 = tf.layers.conv2d(inputs=inputs, filters=50, kernel_size=3,
                             padding="same", activation=tf.nn.relu)
    
    conv2 = tf.layers.conv2d(inputs=conv1, filters=80, kernel_size=5,
                             padding="same", activation=tf.nn.relu)
    
    pool = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
    
    pool_flat = tf.layers.flatten(pool)
    
    
    

In [13]:
x_train[np.where(partition != 1)].shape

(3569, 1000, 20)